<h1> Slippi Data Exploration </h1>

We load required libraries first. The goal at first will be to understand the content of the .slp files in order to identify and isolate relevant features for machine learning applications.

In [26]:
import os as os
import numpy as np
import pandas as pd

import slippi as slp

<h3> Data Loading </h3>

We load a handful of replays. We'll output the characters and winners of each game:

In [32]:
replay_batch = 1000
dataset_path = './Slippi_Public_Dataset_v3/'

# List of all files in the dataset
slp_files = [file for file in os.listdir(dataset_path) if file.endswith('.slp')]

# Load the first three .slp files
for i, slp_file in enumerate(slp_files[:replay_batch]):
    
    # Get file path and store game variable
    file_path = os.path.join(dataset_path, slp_file)
    game = slp.Game(file_path)
    final_frame = game.frames[-1]

    # List occupied ports
    occupied_ports = [i for i, port in enumerate(game.start.players) if port is not None]
    port_1 = occupied_ports[0]
    port_2 = occupied_ports[1]
    
    # Determine winner using final_frame stocks and percentage.
    if (final_frame.ports[port_1].leader.post.stocks < final_frame.ports[port_2].leader.post.stocks or 
            (final_frame.ports[port_1].leader.post.stocks == final_frame.ports[port_2].leader.post.stocks and
              final_frame.ports[port_1].leader.post.damage <= final_frame.ports[port_2].leader.post.stocks)):      # <-- Port priority in action
        winner = port_1
    else:
        winner = port_2

    # Determine characters playing
    port_1_character = game.start.players[port_1].character.name
    port_2_character = game.start.players[port_2].character.name

    print(f"Port {port_1} Character : {port_1_character}")
    print(f"Port {port_2} Character : {port_2_character}")

    print(f'Winner: Port {winner} ({game.start.players[winner].character.name})')



Port 2 Character : FOX
Port 3 Character : CAPTAIN_FALCON
Winner: Port 3 (CAPTAIN_FALCON)
Port 0 Character : CAPTAIN_FALCON
Port 3 Character : FOX
Winner: Port 3 (FOX)
Port 0 Character : FALCO
Port 2 Character : CAPTAIN_FALCON
Winner: Port 0 (FALCO)
Port 0 Character : JIGGLYPUFF
Port 3 Character : SHEIK
Winner: Port 3 (SHEIK)
Port 0 Character : CAPTAIN_FALCON
Port 3 Character : FOX
Winner: Port 0 (CAPTAIN_FALCON)
Port 0 Character : CAPTAIN_FALCON
Port 2 Character : FOX
Winner: Port 2 (FOX)
Port 2 Character : FOX
Port 3 Character : CAPTAIN_FALCON
Winner: Port 3 (CAPTAIN_FALCON)
Port 0 Character : JIGGLYPUFF
Port 3 Character : SHEIK
Winner: Port 0 (JIGGLYPUFF)
Port 0 Character : FALCO
Port 2 Character : CAPTAIN_FALCON
Winner: Port 0 (FALCO)
Port 2 Character : FALCO
Port 3 Character : FOX
Winner: Port 2 (FALCO)
Port 0 Character : FOX
Port 1 Character : SHEIK
Winner: Port 1 (SHEIK)
Port 0 Character : CAPTAIN_FALCON
Port 3 Character : FOX
Winner: Port 0 (CAPTAIN_FALCON)
Port 0 Character : JI

<h3> Frame Data Exploration </h3>

We iterate through the frames of the games and print out the data on player inputs for each frame: 

In [50]:
# Some preliminary functions for printing data and one-hot encoding the button bitmask

def one_hot_encode(bitmask):
    labels = ['DPAD_LEFT', 'DPAD_RIGHT', 'DPAD_DOWN', 'DPAD_UP', 'Z', 'R', 'L', 'A', 'B', 'X', 'Y', 'START']
    encoded_values = [1, 2, 4, 8, 16, 32, 64, 256, 512, 1024, 2048, 4096]

    # Create a dictionary mapping labels to their encoded values
    label_to_value = dict(zip(labels, encoded_values))

    # Initialize a list to store the one-hot encoded values
    one_hot_encoded = [0] * len(labels)

    # Iterate through labels and set the corresponding one-hot encoded value
    for label, value in label_to_value.items():
        if bitmask & value:
            one_hot_encoded[labels.index(label)] = 1

    return one_hot_encoded

def print_player_frame_data(frame, port):
    print(f"\n\tPort {port} Inputs:")
    # print(f"\tLogical Buttons {frame.ports[port].leader.pre.buttons.logical.name}")
    print(f"\tPhysical Buttons {frame.ports[port].leader.pre.buttons.physical}")
    print(f"\tPhysical Buttons Values {frame.ports[port].leader.pre.buttons.physical.value}")
    print(f"\tPhysical Buttons Names {frame.ports[port].leader.pre.buttons.physical.name}")
    # print(f"\tLogical Triggers {frame.ports[port].leader.pre.triggers.logical}")
    # print(f"\tJoystick {frame.ports[port].leader.pre.joystick}")
    print(f"\tJoystick X {frame.ports[port].leader.pre.joystick.x}")
    print(f"\tJoystick Y {frame.ports[port].leader.pre.joystick.y}")
    # print(f"\tC-Stick {frame.ports[port].leader.pre.cstick}")
    print(f"\tC-Stick X {frame.ports[port].leader.pre.cstick.x}")
    print(f"\tC-Stick Y {frame.ports[port].leader.pre.cstick.y}")
    print(f"\tL Trigger {frame.ports[port].leader.pre.triggers.physical.l}")
    print(f"\tR Trigger {frame.ports[port].leader.pre.triggers.physical.r}")

In [33]:
# Physical Buttons are an enumerated class. Note that 128 seems to be missing according to the docs. Will have to verify... (TODO)
# Physical Buttons: (NONE 0, DPAD_LEFT 1, DPAD_RIGHT 2, DPAD_DOWN 4, DPAD_UP 8, Z 16, R 32, L 64, A 256, B 512, X 1024, Y 2048, START 4096)
# Joystick X: [-1, 1]
# Joystick Y: [-1, 1]
# C-Stick X: [-1, 1]
# C-Stick Y: [-1, 1]
# L Trigger: [0, 1]
# R Trigger: [0, 1]
# A one-hot encoding of the physical buttons will give a total of 19 features (13 buttons, 2 joystick, 2 c-stick, 2 triggers)


# unique_physical_buttons = set()

for i, slp_file in enumerate(slp_files[:replay_batch]):
    
    # Get file path and store game variable
    file_path = os.path.join(dataset_path, slp_file)
    game = slp.Game(file_path)

    # List occupied ports
    occupied_ports = [i for i, port in enumerate(game.start.players) if port is not None]
    port_1 = occupied_ports[0]
    port_2 = occupied_ports[1]

    # Get frame data for game
    frames = game.frames

    for i, frame in enumerate(frames):
        print(f"\nFrame {i} Data:")
        print_player_frame_data(frame, port_1)
        print_player_frame_data(frame, port_2)
        # unique_physical_buttons.add(frame.ports[port_1].leader.pre.buttons.physical.value)
        # unique_physical_buttons.add(frame.ports[port_2].leader.pre.buttons.physical.value)

# print(unique_physical_buttons)

{0, 2048, 512, 1024, 3072, 4096, 1536, 2560, 8, 2, 10, 9, 6, 4, 1, 2064, 1040, 16, 2576, 528, 3088, 1552, 3104, 7200, 32, 2080, 544, 1056, 2592, 48, 1072, 560, 3120, 2096, 3136, 64, 2112, 576, 2624, 1088, 4160, 1600, 72, 2049, 1104, 80, 2128, 3152, 2640, 1616, 592, 2144, 1120, 608, 96, 4192, 3168, 112, 3584, 3184, 1136, 4098, 4608, 256, 3328, 2304, 1280, 768, 2816, 4352, 6400, 1792, 3840, 272, 1296, 2320, 784, 3344, 288, 2848, 2336, 4384, 800, 1312, 3360, 304, 1328, 3376, 2352, 320, 2368, 832, 2880, 3392, 4416, 1344, 328, 336, 1360, 2384, 848, 3408, 3424, 864, 2912, 2400, 352, 4448, 1376, 4960, 360, 6496, 2416, 368, 880, 2928, 3440, 1392}
